In this notebook, I show the results of using the equilibrium penalized network in SAM. Following some previous work, I am not applying the neural networks output within a few hundred kilometers of the poleward boundaries.

In [ ]:
import holoviews as hv
hv.extension('bokeh')
%opts Image[width=600, height=400, colorbar=True](cmap='viridis')
%opts Curve[width=400]

# Loading the data

In [ ]:
run_path = "../data/runs/2018-11-09-model188-equilibriation-penalty"
files_3d = f"{run_path}/OUT_3D/*.nc"
files_2d = f"{run_path}/OUT_2D/CASE__1.2Dbin_1.nc"

ds_2d =xr.open_dataset(files_2d)
ds = xr.open_mfdataset(files_3d)

# Some basic plots

Here is the precipitable water:

In [ ]:
%%opts Image{+framewise}
hv.Dataset(ds_2d.PW[::20]).to.image(["x", "y"]).redim.range(PW=(0, 50))

The PW time series is actually pretty promising, but it does diverge around day 112. for some reason.

We can see this in the preciptable water time series

In [ ]:
%%opts Curve[logy=True]
hv.Curve(ds_2d.PW[:,32,0])

And the W500 time series diverges:

In [ ]:
hv.Curve(ds_2d.W500[:,32,0])

I don't think the neural network is causing this problem. Here is the $Q_2$ field at day 112.8, which is right before:

In [ ]:
time_divergence = 112.88

def plot_images_at_time(da, time):
    opts = "Image{+framewise}"
    return hv.Dataset(da.sel(time=time, method='nearest')).to.image(["x", "y"]).opts(opts)

plot_images_at_time(ds.FQTNN[:,:20:2], time_divergence)

There is a large cluster of very strong drying around 10000 km by 6000 km, but this is not that unusally in the data. Notice that $Q_2$ is zero near the meridional boundaries. On the other hand, the vertical velocity field shows some strong artifacts near the boundaries.

In [ ]:
%%opts Image[colorbar=True, width=600]

w_around_blow_up = ds_2d.W500.sel(time=slice(time_divergence-1, time_divergence + .3))[::2]

hv.Dataset(w_around_blow_up).to.image(["x", "y"], dynamic=True).redim.range(W500=(-.2 ,.2))

A couple of things stand out:

1. There is a very fast gibbs like wave propagating west along the southern boundary
2. There is also a 2 Delta x type ringing near the northern boundary which is probably causing he blow.

It is also kind of interesting to look at the absolute vorticity field.

In [ ]:
from uwnet.thermo import coriolis_ngaqua

def vorcitity(u, v):
    f = coriolis_ngaqua(u.y)
    psi = u.differentiate('y') - v.differentiate('x')
    psi.name = 'Vorticity'
    return psi

vort = vorcitity(ds_2d.U850, ds_2d.V850)

In [ ]:
%%opts Image(cmap='RdBu_r')
hv.Dataset(vort[::15]).to.image(["x", "y"], label="Relative Vort")\
.redim.range(Vorticity=(-1e-4, 1e-4))

One thing I note is that all of the cyclones are flung apart at the very start of the simulation, and they are blurred out very quickly by the hyperdiffusion. Then a ton of vorticity accumulates in the southern boundary. Here is the absolute vorticity. Indeed, there is a reversal in at the southern boundary.

In [ ]:
abs_vort = vort + coriolis_ngaqua(vort.y)
abs_vort[140].mean('x').plot()
plt.ylabel("Absolute Vorticity (1/s)");

# Imbalance of the model

## Drift in PW Over the simulation

There is also a substantial drift in the mean moisture. Here is the zonal mean change in PW from the initial condition for three time points.

In [ ]:
pw0 = ds_2d.PW.mean('x')[0]
pw_anom = ds_2d.PW.mean('x') - pw0
pw_anom[[10, 50, 140]].plot(hue='time')

This shows that the tropics are drying out and the subtropics getting much moister.

## Semi-prognostic imbalance

By the way, the model is somewhat imbalanced even when evaluated in semi-prognostic mode:

In [ ]:
import torch

model = torch.load("../models/188/5.pkl")
ds = xr.open_dataset("../data/processed/training.nc")

In [ ]:
output = model.call_with_xr(ds.isel(time=slice(0,None,20), step=0))

In [ ]:
qt_mu = output.QT.mean(['x', 'time'])
fqt_mu = ds.FQT.mean(['x', 'time'])*86400

(fqt_mu+qt_mu).plot()

imbalance = ((fqt_mu+qt_mu)*ds.layer_mass).sum('z')/1000

I really should figure out how to make the scheme predict the right thing in the upper atmosphere.

In [ ]:
imbalance.plot()

This net imbalance field lines up pretty well with the moistening/drying pattern we saw [above](#Drift-in-PW-Over-the-simulation).